In [2]:
import cv2
import numpy as np

## 테스트할 이미지
img_1 = cv2.imread('left.jpg')
img_1 = cv2.resize(img_1, None, fx=0.5, fy=0.5)
gray_1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)
img_2 = cv2.imread('right.jpg')
img_2 = cv2.resize(img_2, None, fx=0.5, fy=0.5)
gray_2 = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)

## 이미지 확인
cv2.imshow('image 1', img_1)
cv2.imshow('image 2', img_2)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [3]:
## 1) 각각의 영상에서 SIFT 특징 추출
#sift 검출기 생성, 각 영상에 대해서 keypoint와 특징 기술자를 뽑아줌
sift = cv2.xfeatures2d.SIFT_create()
kp_1, des_1 = sift.detectAndCompute(gray_1, None)
kp_2, des_2 = sift.detectAndCompute(gray_2, None)

## 2-1) 두 영상의 지역 특징 간 거리 계산
bf = cv2.BFMatcher() 
matches = bf.knnMatch(queryDescriptors = des_1,
                      trainDescriptors = des_2,
                      k=2)  

## 2-2) '최근접 거리 비율' 매칭 전략('ratio testing')을 사용하여 대응점 쌍 생성
ratio = 0.7 
good = []    
for m,n in matches:
    if m.distance < n.distance * ratio:
        good.append(m)

In [4]:
## optional) SIFT 특징점 매칭 결과 확인하기
print(len(good))
matched = cv2.drawMatches(img1 = img_1, 
                          keypoints1 = kp_1, 
                          img2 = img_2, 
                          keypoints2 = kp_2, 
                          matches1to2 = good[:20],   # 첫 20개 매칭쌍만 시각화
                          outImg = None, 
                          flags = 2)

cv2.imshow('matching result', matched)
cv2.waitKey(0)

cv2.destroyAllWindows()

243


In [5]:
## 3) RANSAC을 이용한 변환 행렬 H 추정
if len(good) >= 4:   # 4개 이상의 대응점이 존재해야 원근 변환 행렬을 추정 가능
    src_pts = np.float32([ kp_2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    H, _ = cv2.findHomography(srcPoints = src_pts, #3차원의 물체를 원근변환해주는 것을 의미
                              dstPoints = dst_pts, 
                              method = cv2.RANSAC, #RANSAC을 이용함 --> RANSAC을 기반으로 변환행렬 추적
                              ransacReprojThreshold = 5) 

    
else:
    raise AssertionError("Can't find enough keypoints.")

### Homework ( ~ 12/02 12:00)
- 아래 코드의 '4) 원근 변환 적용', '5) 두 영상을 한 장으로 이어붙이기' 부분을 완성하여 파노라마 영상 만들기

In [8]:
## 4) 원근 변환 적용
img_3 = cv2.warpPerspective(img_2, H, (img_1.shape[1] * 2, img_1.shape[0]))

for i in range(img_1.shape[0]):
    for j in range(img_1.shape[1]):
        #img_3[i][j] = img_1[i][j]
        img_3[i][j] = img_1[i][j].copy()
res = img_3

## 결과 파노라마 영상 시각화
cv2.imshow('panorama', res)
cv2.waitKey(0)

cv2.destroyAllWindows()